In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.io.vasp import Poscar
from pymatgen.util.coord import pbc_diff
from pymatgen.core import Structure

%matplotlib inline
plt.rcParams["font.family"] = "Helvetica"

def objective_function(order, structure_A, structure_B, lattice):
    ordered_structure_B = structure_B[order]
    distance_sum = 0
    for i in range(len(structure_A)):
        frac_distance = pbc_diff(structure_A[i], ordered_structure_B[i])
        cart_distance = lattice.get_cartesian_coords(frac_distance)
        distance_sum += np.dot(cart_distance, cart_distance)
    return distance_sum

def simulated_annealing(initial_order, structure_A, structure_B, lattice, temperature=1.0, cooling_rate=0.99999, num_iterations=200000):
    current_order = initial_order.copy()
    best_order = current_order.copy()
    current_distance = objective_function(current_order, structure_A, structure_B, lattice)
    best_distance = current_distance

    objective_values = []

    for iteration in range(num_iterations):
        # Generate a neighboring order by randomly swapping two indices
        neighbor_order = current_order.copy()
        i, j = np.random.choice(len(current_order), size=2, replace=False)
        neighbor_order[i], neighbor_order[j] = neighbor_order[j], neighbor_order[i]

        # Calculate the distance for the neighboring order
        neighbor_distance = objective_function(neighbor_order, structure_A, structure_B, lattice)

        # Accept the neighboring order with a probability determined by the Metropolis criterion
        if neighbor_distance < current_distance or np.random.rand() < np.exp((current_distance - neighbor_distance) / temperature):
            current_order = neighbor_order
            current_distance = neighbor_distance

        # Update the best order if the current order is an improvement
        if current_distance < best_distance:
            best_order = current_order
            best_distance = current_distance

        # Reduce the temperature
        temperature *= cooling_rate

        # Save the objective function value every 1000 iterations
        if iteration % int(num_iterations/100) == 0:
            objective_values.append(current_distance)

    return best_order, objective_values

In [ ]:
# Read atomic structures from POSCAR file
# POSCARs contain ONLY Ga atoms!
folder_path = "Simulated_Annealing"
poscar_file_A = folder_path + '/CONTCAR_B_tuned.vasp'
poscar_file_B = folder_path + '/CONTCAR_G_tuned.vasp'

structure_A = Poscar.from_file(poscar_file_A).structure.frac_coords
structure_B = Poscar.from_file(poscar_file_B).structure.frac_coords
lattice_A = Poscar.from_file(poscar_file_A).structure.lattice

plt.figure(figsize=(4,3), dpi=200)

min_dis = np.inf
for i in range(1,20):
    
    # Initialize with the identity permutation
    initial_order = np.random.permutation(len(structure_A))
        
    # Apply simulated annealing to find the optimal permutation
    best_order, objective_values = simulated_annealing(initial_order, structure_A, structure_B, lattice_A)
    num_iterations = 5000
    # Plotting the objective function values over iterations
    iterations = range(0, num_iterations, int(num_iterations/100))
    plt.plot(iterations, objective_values, linestyle='-',lw=0.75)
    plt.xlabel('Iteration')
    plt.ylabel('$\Sigma $ displacements$^{2}$')
    plt.yscale("log")  
    
    if min_dis > objective_function(best_order, structure_A, structure_B, lattice_A):
        min_dis = objective_function(best_order, structure_A, structure_B, lattice_A)
        print(f"{i:02d}th Minimum Distance Sum: {objective_function(best_order, structure_A, structure_B, lattice_A)}")


        # Create a Structure object with the reordered Cartesian structure_B and the lattice of structure_B
        lattice_B = Poscar.from_file(poscar_file_B).structure.lattice
        reordered_structure_B_cartesian = structure_B[best_order]
        reordered_structure_B = Structure(lattice=lattice_B,
                                              species=["Ga"] * len(reordered_structure_B_cartesian),
                                              coords=reordered_structure_B_cartesian,coords_are_cartesian=False)

        # Write the reordered structure to a POSCAR file
        poscar_file_reordered_torch = folder_path + f'/Results/Test_CONTCAR_out{i,round(objective_function(best_order, structure_A, structure_B, lattice_A), 2)}.vasp'
        Poscar(reordered_structure_B).write_file(poscar_file_reordered_torch)
    else:
        continue

In [ ]:
# Read atomic structures from POSCAR file
# POSCARs contain ONLY Ga atoms!
folder_path = "Simulated_Annealing"
poscar_file_A = folder_path + '/CONTCAR_B_VGa_tuned.vasp'
poscar_file_B = folder_path + '/CONTCAR_G_VGa_tuned.vasp'

structure_A = Poscar.from_file(poscar_file_A).structure.frac_coords
structure_B = Poscar.from_file(poscar_file_B).structure.frac_coords
lattice_A = Poscar.from_file(poscar_file_A).structure.lattice

plt.figure(figsize=(4,3), dpi=200)

min_dis = np.inf
for i in range(1,20):
    
    # Initialize with the identity permutation
    initial_order = np.random.permutation(len(structure_A))
        
    # Apply simulated annealing to find the optimal permutation
    best_order, objective_values = simulated_annealing(initial_order, structure_A, structure_B, lattice_A)
    num_iterations = 5000
    # Plotting the objective function values over iterations
    iterations = range(0, num_iterations, int(num_iterations/100))
    plt.plot(iterations, objective_values, linestyle='-',lw=0.75)
    plt.xlabel('Iteration')
    plt.ylabel('$\Sigma $ displacements$^{2}$')
    plt.yscale("log")  
    
    if min_dis > objective_function(best_order, structure_A, structure_B, lattice_A):
        min_dis = objective_function(best_order, structure_A, structure_B, lattice_A)
        print(f"{i:02d}th Minimum Distance Sum: {objective_function(best_order, structure_A, structure_B, lattice_A)}")
        num_iterations = 1000
        # Plotting the objective function values over iterations
        iterations = range(0, num_iterations, int(num_iterations/100))
        plt.plot(iterations, objective_values, linestyle='-',lw=0.75)
        plt.xlabel('Iteration')
        plt.ylabel('$\Sigma $ displacements$^{2}$')
        plt.yscale("log")  

        # Create a Structure object with the reordered Cartesian structure_B and the lattice of structure_B
        lattice_B = Poscar.from_file(poscar_file_B).structure.lattice
        reordered_structure_B_cartesian = structure_B[best_order]
        reordered_structure_B = Structure(lattice=lattice_B,
                                              species=["Ga"] * len(reordered_structure_B_cartesian),
                                              coords=reordered_structure_B_cartesian,coords_are_cartesian=False)

        # Write the reordered structure to a POSCAR file
        poscar_file_reordered_torch = folder_path + f'/Results_VGa_/Test_CONTCAR_out{i,round(objective_function(best_order, structure_A, structure_B, lattice_A), 2)}.vasp'
        Poscar(reordered_structure_B).write_file(poscar_file_reordered_torch)
    else:
        continue